In [32]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

from IPython import display

from dotenv import load_dotenv


In [33]:
load_dotenv()

True

In [30]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [34]:
os.getenv("OPENAI_API_KEY")

'sk-proj-wrUaZtPR4Re7jQhJj7foT3BlbkFJjb2akhjXkKdQRYYd9V6i'

In [20]:
text_model =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=1500,
)

In [21]:
def extract_images_and_tables_from_unstructured_pdf(pdf_path,output_path):
    # Load the PDF file
    raw_pdf_elements=partition_pdf(
    filename=pdf_path,                  
    strategy="hi_res",                                
    extract_images_in_pdf=True,                      
    extract_image_block_types=["Image", "Table"],          
    extract_image_block_to_payload=False,                  
    extract_image_block_output_dir=output_path, 
    )
    
    img=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

    tab=[]
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

    return img,tab


In [22]:

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [23]:
def summarize_image(encode_image , model):
    response = client.chat.t

In [24]:
def summarize_image(encoded_image, vision_model):
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    msg = [
        HumanMessage(content=[
            {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = vision_model.invoke(msg)
    return response.content

In [25]:
encoded_image = encode_image("Capture6.PNG")

In [26]:
response = summarize_image(encoded_image, text_model)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************9V6i. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [1]:
!pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/33/53/c15e373e6f8057afb0e73a5bd4be85069252d7daa1dc09d21272495e8657/boto3-1.35.31-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.36.0,>=1.35.31 from https://files.pythonhosted.org/packages/c7/b8/af6ff2c29315c0be0dcdec98ba42bb699e531bca83fbab1027fee060312e/botocore-1.35.31-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/3c/4a/b221409913760d26cf4498b7b1741d510c82d3ad38381984a3ddc135ec66/s3transfer-0.10.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/139.1 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.1 kB 1.4 MB/s eta 0:00


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#upload the image and get the url
import boto3
import botocore
import os



def upload_to_s3(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name

    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        print(e)
        return False
   
    return True

def download_from_s3(bucket, object_name, file_name):
    s3 = boto3.client('s3')
    try:
        s3.download_file(bucket, object_name, file_name)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

def get_url(bucket, object_name):
    return f"https://{bucket}.s3.amazonaws.com/{object_name}"




In [2]:
upload_to_s3(".\\images\\1a4ac1e572454afcabc53a966f0b6da5figure-6-2.jpg", "obotutor", "image1.jpeg")

True

In [9]:
get_url("obotutor", "image1.jpeg")

'https://obotutor.s3.amazonaws.com/image1.jpeg'

In [3]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader

def get_pdf_text(pdf):
    text = ""

    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [4]:
get_pdf_text("saved_pdfs\copy_Python Operators (With Examples).pdf")

"Python Operators\nOperators are special symbols that perform operations on\nvariables (/python-programming/variables-constants-\nliterals) and values. For example,\nRun Code (/python-programming/online-compiler)\nHere, + is an operator that adds two numbers: 5 and 6.\nTypes of Python Operators\nHere's a list of different types of Python operators that we\nwill learn in this tutorial.\n\x00. Arithmetic Operators\n\x00. Assignment Operators\n\x00. Comparison Operators\n\x00. Logical Operators\n\x00. Bitwise Operators\n\x00. Special Operatorsprint(5 + 6)   # 11\nThank you for printing our content at www .domain-name.com. Please check back soon for new contents.\nwww.domain-name.com(h\x00ps://programiz.pro/learn/master-python?\nutm_source=sticky-\nbanner&utm_campaign=programiz&utm_medium=referral)36%\noffLearn to code solving problems with our\nhands-on Python course! Try Programiz PRO\ntoday.\n(/)\nSearch tutorials & examplesProgramiz\nPRO(h\x00ps://programiz.pro/learn/master-python?\nut